# <Step 1> 크롤링 : 웹 데이터 가져오기

### [BeautifulSoup을 이용해 웹 크롤링 하기]

In [6]:
# pip install selenium beautifulsoup4 로 라이브러리 설치
# pip install webdriver_manager로 라이브러리 설치
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import re
# pip uninstall konlpy==0.5.1 jpype1==0.6.3 Jpype1-py3
# pip install konlpy==0.5.1 jpype1==0.6.3 Jpype1-py3
#pip install konlpy==0.5.1
# pip install jpype==0.6.3
# pip install Jpype1-py3
from konlpy.tag import Okt
from collections import Counter


# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://namu.wiki/RecentChanges"

#options = webdriver.ChromeOptions()
#options.add_experimental_option("excludeSwitches", ["enable-logging"])
# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
#driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)  # for Windows
driver = webdriver.Chrome(ChromeDriverManager().install())  # for Windows
driver.get(source_url)
driver.implicitly_wait(10)
#table_rows = driver.find_elements(By.XPATH,'//*[@id="C6Rc9QlVe"]/div[2]/div/div/div/div/div/article/div[3]/div/div/div/div[1]/div/div/table/tbody/tr/td/a')
table_rows = driver.find_elements(By.XPATH,'//*[@id="C6Rc9QlVe"]/div[2]/div/div/div/div/div/article/div[3]/div/div/div/div[1]/div/div/table/tbody/tr')
print(table_rows)
page_urls = []

#for i in range(0,len(table_rows)): # 시간이 너무 오래 걸림
for i in range(0,5):    # 어차피 맨 밑에 상위 다섯번째까지만 출력하면 되니 5번만 반복.
    first_td=table_rows[i].find_elements(By.TAG_NAME,"td")
    td_url=first_td[0].find_elements(By.TAG_NAME,"a")
    if len(td_url) >0 :
        page_url=td_url[0].get_attribute("href")
        #page_url=table_rows[0].get_attribute("href")   # a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
        print(page_url)
        if 'png' not in page_urls:
            page_urls.append(page_url)

# 중복 url을 제거합니다.
page_urls = list(set(page_urls))
print(page_urls)
#for page in page_urls[:3]:
#    print(page)





C:\Users\GR\AppData\Local\Temp\ipykernel_10304\421460542.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())  # for Windows


[<selenium.webdriver.remote.webelement.WebElement (session="de263a30f167b1208a339ab36a1d8bdc", element="e8edfb75-47cd-4217-9b56-1dff20eafdf5")>, <selenium.webdriver.remote.webelement.WebElement (session="de263a30f167b1208a339ab36a1d8bdc", element="616feac3-de8d-41c5-92f4-df89fbca1f0a")>, <selenium.webdriver.remote.webelement.WebElement (session="de263a30f167b1208a339ab36a1d8bdc", element="1d3b6cc5-c527-4824-9e4f-4eeb758c14aa")>, <selenium.webdriver.remote.webelement.WebElement (session="de263a30f167b1208a339ab36a1d8bdc", element="13bfbe55-f482-4f5a-826d-a7e1c479790c")>, <selenium.webdriver.remote.webelement.WebElement (session="de263a30f167b1208a339ab36a1d8bdc", element="693b6a53-882b-4890-8b94-70a9e01b8478")>, <selenium.webdriver.remote.webelement.WebElement (session="de263a30f167b1208a339ab36a1d8bdc", element="afbc929a-bd57-49a8-be0f-1418c6873d3f")>, <selenium.webdriver.remote.webelement.WebElement (session="de263a30f167b1208a339ab36a1d8bdc", element="485b6e7c-75f1-46a0-ad2f-c3da011c

### [나무위키의 최근 변경 데이터 크롤링하기]

In [7]:

columns = ['title','category','content_text']
df=pd.DataFrame(columns=columns)

for page_url in page_urls:
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(10)
    driver.get(page_url)
    req=driver.page_source
    soup=BeautifulSoup(req,"html.parser")
    contents_table=soup.find(attrs={"class":"SVuqC-pU"})
    title=contents_table.find_all("h1")[0]

    if len(contents_table.find_all("ul"))>0:
        category=contents_table.find_all("ul")[0]
    else:
        category=None

    content_paragraphs=contents_table.find_all(name="div", attrs={"class":"UtCm7-qJ"})
    content_corpus_list=[]

    if title is not None:
        row_title=title.text.replace("\n"," ")
    else:
        row_title=""
    
    if content_paragraphs is not None:
        for paragraghs in content_paragraphs:
            if paragraghs is not None:
                content_corpus_list.append(paragraghs.text.replace("\n"," "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list("")

    if category is not None:
        row_category=category.text.replace("\n"," ")
    else:
        row_category=""

    row=[row_title,row_category,"".join(content_corpus_list)]
    series=pd.Series(row, index=df.columns)
    df=df.append(series, ignore_index=True)




    driver.close()
df.head(5)

C:\Users\GR\AppData\Local\Temp\ipykernel_10304\2721062881.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\GR\AppData\Local\Temp\ipykernel_10304\2721062881.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(series, ignore_index=True)


,title,category,content_text
0,러시아 프리미어 리그/2021-22 시즌,러시아 프리미어 리그,2021-22시즌 러시아 프리미어 리그는 소련 해체 이후 30번째 시즌이다.2021...
1,애산 전투,중국사/전투몽골-남송 전쟁,“충후함으로써 전대의 자손을 기르고(以忠厚養前代之子孫)관대함으로써 사인의 정기를 기...
2,2022 FIFA 월드컵 카타르/E조/독일 vs 일본,2022년 축구 경기2022 FIFA 월드컵 카타르독일 축구 국가대표팀 경기일본 축...,2022 FIFA 월드컵 카타르 E조 1경기2022년 11월 23일 수요일 22:0...
3,이해진,대한민국의 남성 기업인1967년 출생전주 이씨 덕흥대원군파동대문구 출신 인물삼성SD...,"대한민국의 기업인. 네이버의 창업자, 글로벌투자책임자.1967년 6월 22일, 서울..."
4,트런들,리그 오브 레전드/챔피언,"""이길 수 없으면 속이고, 속일 수 없으면 이겨라.""트런들은 몸집이 크고 교활한데다..."


### [텍스트 데이터 전처리 하기]

In [8]:
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

print(text_cleaning(df["content_text"][0]))

시즌 러시아 프리미어 리그는 소련 해체 이후 번째 시즌이다년 월 일에 시작해 년 월 일에 끝날 예정이다참가 클럽전 시즌 순위특이사항리그 첫 참가최근 승격 시즌 제니트 상트페테르부르크우승 조별 리그 진출 스파르타크 모스크바준우승 차예선 진출 로코모티프 모스크바위 조별리그 진출 루빈 카잔위 차예선 진출 소치위 차예선 진출  모스크바위 디나모 모스크바위 힘키위 로스토프위 크라스노다르위 아흐마트 그로즈니위 우랄 예카테린부르크위 우파위승강 플레이오프 미실시로 잔류 아르세날 툴라위승강 플레이오프 미실시로 잔류 크릴리야 소베토프 사마라풋볼 내셔널 리그부 우승 니즈니 노브고로드풋볼 내셔널 리그부 위팀 중 소련 해체 이후 창단된 팀이 팀이다순위팀순위팀위 탐보프위 크릴리야 소베토프 사마라위 로토르 볼고그라드위 니즈니 노브고로드부리그 위 오렌부르크와 위 알라니야 블라디캅카스가 프리미어 리그 라이센스 획득에 실패하면서 승강 플레이오프가 일어나지 않았고 부리그 위와 위가 승격하게 되었다  탐보프는 해체의 길을 걷게 된다   시즌 클럽별 감독  펼치기  접기 슬라비샤요카노비치디나모 모스크바발레리카르핀로스토프미하일갈락티오노프로코모티프 모스크바알렉산드르토칠린형식상소치기예르모아바스칼스파르타크 모스크바세르게이타슈예프아흐마트마르첼리치카오렌부르크빅토르곤차렌코우랄세르게이세마크제니트블라디미르페도토프 모스크바알렉산드르스토로주크크라스노다르이고리오신킨크릴리야 소베토프니콜라이사비체프형식상토르페도 모스크바미하일갈락티오노프파리 올레크바실렌코파켈니콜라이피사레프힘키러시아 풋볼 내셔널 리그 감독 보기  프리미어리그   프리미어리그   프리미어리그전세계 리그 및 대표팀 감독 보기형식상 라이센스 문제로 형식상으로만 감독에 등재 사실상 감독은 코치인 바딤 가라닌형식상 라이센스 문제로 형식상으로만 감독에 등재 사실상 감독은 코치인 니콜라이 코바르다예프구단전임 감독사유해지일당시 순위신임 감독선임일당시 순위 스파르타크 모스크바 도메니코 테데스코계약 만료프리시즌 후이 비토리아프리시즌  모스크바 이비차 올리치경질프리시즌 알렉세이

### [모든 데이터 전처리 적용하기]

In [9]:
df["title"]=df["title"].apply(lambda x:text_cleaning(x))
df["category"]=df["category"].apply(lambda x:text_cleaning(x))
df["content_text"]=df["content_text"].apply(lambda x:text_cleaning(x))

print(df.head(5))


               title                                           category  \
0    러시아 프리미어 리그 시즌                                         러시아 프리미어 리그   
1             애산 전투                                        중국사전투몽골남송 전쟁   
2    월드컵 카타르조독일  일본        년 축구 경기  월드컵 카타르독일 축구 국가대표팀 경기일본 축구 국가대표팀 경기   
3               이해진   대한민국의 남성 기업인년 출생전주 이씨 덕흥대원군파동대문구 출신 인물삼성의 인물네이...   
4               트런들                                        리그 오브 레전드챔피언   

                                        content_text  
0  시즌 러시아 프리미어 리그는 소련 해체 이후 번째 시즌이다년 월 일에 시작해 년 월...  
1  충후함으로써 전대의 자손을 기르고관대함으로써 사인의 정기를 기르며절제함으로써 백성의...  
2    월드컵 카타르 조 경기년 월 일 수요일  한국시각칼리파 인터내셔널 스타디움 카타...  
3  대한민국의 기업인 네이버의 창업자 글로벌투자책임자년 월 일 서울특별시 동대문구 제기...  
4  이길 수 없으면 속이고 속일 수 없으면 이겨라트런들은 몸집이 크고 교활한데다 잔인한...  


### [말뭉치 만들기] 

In [10]:
title_corpus="".join(df["title"].tolist())
category_corpus="".join(df["category"].tolist())
content_corpus="".join(df["content_text"].tolist())

print(title_corpus)
print(category_corpus)
print(content_corpus)

러시아 프리미어 리그 시즌 애산 전투   월드컵 카타르조독일  일본 이해진 트런들 
러시아 프리미어 리그중국사전투몽골남송 전쟁년 축구 경기  월드컵 카타르독일 축구 국가대표팀 경기일본 축구 국가대표팀 경기대한민국의 남성 기업인년 출생전주 이씨 덕흥대원군파동대문구 출신 인물삼성의 인물네이버기업상문고등학교 출신서울대학교 출신한국과학기술원 출신대한민국의 프로그래머리그 오브 레전드챔피언
시즌 러시아 프리미어 리그는 소련 해체 이후 번째 시즌이다년 월 일에 시작해 년 월 일에 끝날 예정이다참가 클럽전 시즌 순위특이사항리그 첫 참가최근 승격 시즌 제니트 상트페테르부르크우승 조별 리그 진출 스파르타크 모스크바준우승 차예선 진출 로코모티프 모스크바위 조별리그 진출 루빈 카잔위 차예선 진출 소치위 차예선 진출  모스크바위 디나모 모스크바위 힘키위 로스토프위 크라스노다르위 아흐마트 그로즈니위 우랄 예카테린부르크위 우파위승강 플레이오프 미실시로 잔류 아르세날 툴라위승강 플레이오프 미실시로 잔류 크릴리야 소베토프 사마라풋볼 내셔널 리그부 우승 니즈니 노브고로드풋볼 내셔널 리그부 위팀 중 소련 해체 이후 창단된 팀이 팀이다순위팀순위팀위 탐보프위 크릴리야 소베토프 사마라위 로토르 볼고그라드위 니즈니 노브고로드부리그 위 오렌부르크와 위 알라니야 블라디캅카스가 프리미어 리그 라이센스 획득에 실패하면서 승강 플레이오프가 일어나지 않았고 부리그 위와 위가 승격하게 되었다  탐보프는 해체의 길을 걷게 된다   시즌 클럽별 감독  펼치기  접기 슬라비샤요카노비치디나모 모스크바발레리카르핀로스토프미하일갈락티오노프로코모티프 모스크바알렉산드르토칠린형식상소치기예르모아바스칼스파르타크 모스크바세르게이타슈예프아흐마트마르첼리치카오렌부르크빅토르곤차렌코우랄세르게이세마크제니트블라디미르페도토프 모스크바알렉산드르스토로주크크라스노다르이고리오신킨크릴리야 소베토프니콜라이사비체프형식상토르페도 모스크바미하일갈락티오노프파리 올레크바실렌코파켈니콜라이피사레프힘키러시아 풋볼 내셔널 리그 감독 보기  프리미어리그   프리미어리그   프리

### [konlpy를 이용해 키워드 추출하기]

In [11]:
!pip install konlpy==0.5.1
!pip install jpype1==0.6.3
!pip install Jpype1-py3

In [12]:
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)
print(count)
remove_char_counter=Counter({x : count[x] for x in count if len(x)>1})
print(remove_char_counter)

Counter({'트런들': 404, '것': 189, '수': 178, '독일': 142, '년': 126, '때문': 123, '경기': 117, '스킬': 114, '팀': 112, '정글': 112, '일본': 111, '이': 108, '상대': 101, '탱커': 98, '챔피언': 95, '공격': 87, '리그': 84, '전': 84, '번': 83, '기둥': 83, '적': 82, '때': 82, '그': 78, '등': 75, '시즌': 74, '차': 74, '효과': 73, '이전': 72, '체력': 72, '위': 71, '월드컵': 70, '패': 68, '능력': 66, '월': 65, '경우': 64, '정도': 64, '아이템': 64, '모스크바': 62, '중': 61, '문서': 60, '상황': 60, '탑': 59, '이후': 57, '더': 57, '역사': 57, '판': 56, '패치': 56, '로': 55, '문단': 55, '증가': 54, '대회': 53, '기': 52, '라인': 52, '의': 51, '일': 48, '또한': 48, '속도': 48, '얼음': 48, '를': 46, '시작': 45, '대한민국': 43, '선수': 42, '초': 42, '방어': 41, '상대로': 40, '타': 39, '승률': 39, '사용': 38, '안': 38, '명': 37, '점': 37, '버프': 37, '너프': 37, '승리': 36, '기록': 36, '둔화': 36, '당시': 35, '볼': 35, '선택': 35, '감소': 35, '티어': 35, '러시아': 34, '사실': 34, '말': 34, '강': 34, '레벨': 34, '네이버': 34, '피해': 34, '궁극': 34, '아군': 34, '점수': 33, '두': 33, '모두': 33, '시간': 33, '시브': 33, '초반': 32, '딜러': 32, '하나': 31, '도': 31, '킹': 31, '서

### [키워드 가다듬기 : 불용어 사전] 

In [13]:
# 한국어 약식 불용어사전 예시 파일입니다. 출처 - (https://www.ranks.nl/stopwords/korean)
korean_stopwords_path = "/Data.python/python-data-analysis-master/data/korean_stopwords.txt"

# 텍스트 파일을 오픈합니다.
with open(korean_stopwords_path,encoding="utf-8")as f :
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]
print(stopwords[:10])

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']


### [키워드 가다듬기 : 불용어 제거]

In [14]:
# 나무위키 페이지에 맞는 불용어를 추가합니다.
namu_wiki_stopwords=['상위', '문서', '내용', '누설', '아래', '해당', '설명', '표기', '추가', '모든', '사용', '매우', '가장',
                       '줄거리', '요소', '상황', '편집', '틀', '경우', '때문', '모습', '정도', '이후', '사실', '생각', '인물', 
                       '이름', '년월']
for stopword in namu_wiki_stopwords:
    stopwords.append(stopword)
    
# 키워드 데이터에서 불용어를 제거합니다.
remove_char_counter = Counter({x : remove_char_counter[x] for x in count if x not in stopwords})
print(remove_char_counter)

Counter({'트런들': 404, '독일': 142, '경기': 117, '스킬': 114, '정글': 112, '일본': 111, '상대': 101, '탱커': 98, '챔피언': 95, '공격': 87, '리그': 84, '기둥': 83, '시즌': 74, '효과': 73, '이전': 72, '체력': 72, '월드컵': 70, '능력': 66, '아이템': 64, '모스크바': 62, '역사': 57, '패치': 56, '문단': 55, '증가': 54, '대회': 53, '라인': 52, '속도': 48, '얼음': 48, '시작': 45, '대한민국': 43, '선수': 42, '방어': 41, '상대로': 40, '승률': 39, '버프': 37, '너프': 37, '승리': 36, '기록': 36, '둔화': 36, '당시': 35, '선택': 35, '감소': 35, '티어': 35, '러시아': 34, '레벨': 34, '네이버': 34, '피해': 34, '궁극': 34, '아군': 34, '점수': 33, '시브': 33, '초반': 32, '딜러': 32, '서포터': 31, '마법': 30, '감독': 28, '성장': 28, '리메이크': 28, '갱킹': 28, '안티': 28, '카잔': 27, '결과': 27, '전투': 27, '챔프': 27, '지속': 26, '한국': 26, '스페인': 26, '진압': 26, '기준': 25, '합산': 25, '남송': 25, '카운터': 25, '평가': 25, '트롤': 25, '교전': 25, '아시아': 24, '덕분': 24, '예선': 23, '문제': 23, '아레나': 23, '수준': 23, '기적': 23, '회복': 23, '장판': 23, '메타': 23, '브루': 22, '통해': 22, '애산': 22, '더욱': 22, '정글러': 22, '결정': 21, '이기': 21, '역할': 21, '다시': 21, '크게': 21, '치명': 21, '제니트'

-----

# <Step 3> 시각화 : 워드클라우드 시각화하기

### [pytagcloud 사용하기]

- 아래 코드 실행을 위해, anaconda prompt 혹은 Terminal에서 아래와 같은 패키지들을 설치해 줍니다.
    - (env_name) `pip install pytagcloud pygame simplejson`
- 그리고 아래와 같은 경로에 한글 폰트(예: NanumBarunGothic.ttf) 파일을 옮깁니다. 
    - Mac OS : /{anaconda_path}/envs/{env_name}/lib/python3.8/site-packages/pytagcloud/fonts
    - Windosw OS : \{anaconda_path}\envs\{env_name}\Lib\site-packages\pytagcloud\fonts
    - `폰트 다운로드 : http://hangeul.naver.com/webfont/NanumGothic/NanumGothic.ttf`
- 파일을 옮긴 후, 파이썬 가상환경을 재실행 하여 주피터를 다시 실행해줍니다. 그래도 아래의 코드가 실행되지 않는 경우는 다음과 같은 작업을 수행합니다.
    - 위의 경로에서 font.json 파일을 편집합니다.
    - 아래와 같은 코드를 추가하고 font.json 파일을 저장합니다.
    - {
                "name": "NanumGothic",
                "ttf": "NanumGothic.ttf",
                "web": "http://fonts.googleapis.com/css?family=Nanum+Gothic"
        },

In [15]:
pip install pytagcloud pygame simplejson

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import random
import pytagcloud
import webbrowser

# 가장 출현 빈도수가 높은 40개의 단어를 선정합니다.
ranked_tags = remove_char_counter.most_common(40)

# pytagcloud로 출력할 40개의 단어를 입력합니다. 단어 출력의 최대 크기는 80으로 제한합니다.
taglist = pytagcloud.make_tags(ranked_tags, maxsize=80)

# pytagcloud 이미지를 생성합니다. 폰트는 나눔 고딕을 사용합니다.
pytagcloud.create_tag_image(taglist, 'wordcloud.jpg', size=(900, 600), fontname='NanumGothic', rectangular=False)

In [ ]:
# 생성한 이미지를 주피터 노트북상에서 출력합니다.
from IPython.display import Image
Image(filename='wordcloud.jpg')

-----

### [나무위키 키워드 시각화]

In [ ]:
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(title_corpus)
count = Counter(nouns)

remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})
remove_char_counter = Counter({x : remove_char_counter[x] for x in count if x not in stopwords})

ranked_tags = remove_char_counter.most_common(40)
taglist = pytagcloud.make_tags(ranked_tags, maxsize=80)
pytagcloud.create_tag_image(taglist, 'title_wordcloud.jpg', size=(900, 600), fontname='NanumGothic', rectangular=False)

Image(filename='title_wordcloud.jpg')

##### 제목 키워드

-----

##### 카테고리 키워드

In [ ]:
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(category_corpus)
count = Counter(nouns)

remove_char_counter = Counter({x : count[x] for x in count if len(x) > 1})
remove_char_counter = Counter({x : remove_char_counter[x] for x in count if x not in stopwords})

ranked_tags = remove_char_counter.most_common(40)
taglist = pytagcloud.make_tags(ranked_tags, maxsize=80)
pytagcloud.create_tag_image(taglist, 'category_wordcloud.jpg', size=(900, 600), fontname='NanumGothic', rectangular=False)

Image(filename='category_wordcloud.jpg')